In [9]:
!pip install onnx==1.13.1 onnxruntime skl2onnx pandas scikit-learn
import logging
import sys
import warnings
import onnx
from urllib.parse import urlparse

You should consider upgrading via the '/Users/tyler.yang/Downloads/jupyter/bin/python3 -m pip install --upgrade pip' command.


In [10]:
import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn import pipeline
from sklearn import linear_model
from urllib.parse import urlparse
from sklearn.preprocessing import FunctionTransformer

In [11]:
!pip install mlflow

You should consider upgrading via the '/Users/tyler.yang/Downloads/jupyter/bin/python3 -m pip install --upgrade pip' command.


In [12]:
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

/Users/tyler.yang/Downloads/jupyter/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [25]:
from skl2onnx import to_onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

In [26]:
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [27]:
def convert_to_onnx(model, data):
    #check the specifications
    initial_types = list(
    zip(
        data.columns.values.tolist(),
        [FloatTensorType([None, 1]) for _ in range(len(data.columns))],
    )
)
    onnx_model = convert_sklearn(model=model, initial_types=initial_types)
    print("onnx_model.type:",type(onnx_model))
    mlflow.set_tag("onnx_version",onnx.__version__)
    return onnx_model

In [28]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [29]:
warnings.filterwarnings("ignore")
np.random.seed(40)

Read the wine-quality csv file from the URL

In [30]:
csv_url = (
        "http://archive.ics.uci.edu/ml"
        "/machine-learning-databases/wine-quality/winequality-red.csv"
    )
try:
    data = pd.read_csv(csv_url, sep=";")
except Exception as e:
    logger.exception(
        "Unable to download training & test CSV, check your internet connection. Error: %s", e
    )

Split the data into training and test sets. (0.75, 0.25) split.

In [31]:
train, test = train_test_split(data)

The predicted column is "quality" which is a scalar from [3, 9]

In [33]:
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

In [34]:
alpha = 0.5
l1_ratio = 0.5

In [48]:
with mlflow.start_run():
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    DummyScaler = FunctionTransformer(None)
    lr = pipeline.Pipeline(
    [("dummy", DummyScaler), ("reg", linear_model.ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42))]
)
    lr.fit(train_x, train_y)
    predicted_qualities = lr.predict(test_x)
    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)
    print(f"Elasticnet model (alpha={alpha:f}, l1_ratio={l1_ratio:f}):")
    print(f"  RMSE: {rmse}")
    print(f"  MAE: {mae}")
    print(f"  R2: {r2}")
    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)
    predictions = lr.predict(train_x)
    model_signature = infer_signature(train_x, predictions)
    onnx_model = convert_to_onnx(lr, test_x)
    mlflow.onnx.log_model(onnx_model, "model", registered_model_name="ElasticnetWineModel", signature=model_signature)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.7931640229276851
  MAE: 0.6271946374319586
  R2: 0.10862644997792614
onnx_model.type: <class 'onnx.onnx_ml_pb2.ModelProto'>


Registered model 'ElasticnetWineModel' already exists. Creating a new version of this model...
Created version '10' of model 'ElasticnetWineModel'.
